In [176]:
city = "Abbotsford"
fav_cuisine = ["Italian", "Pizza"]
rest_name = ["Koya Japan", "An Indian Affair"]
order_pref = ['food' , 'service'	, 'value']
usid = 100
lat = 49.11562
lon = -122.67953

In [224]:
!pip install pymysql

     |████████████████████████████████| 51kB 2.6MB/s 


In [4]:
import pymysql
import pandas as pd
connection = pymysql.connect(host="sql395.main-hosting.eu",
                             user="u882629798_nipungoel",
                             passwd="wHAT@2314",
                             db="u882629798_TravelBack",
                             charset='utf8mb4', 
                             cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

In [5]:
SQL_Query = pd.read_sql_query(
'''select *
from CLEANED_BRITISH_RES''', connection)
d = pd.DataFrame(SQL_Query)

SQL_Query1 = pd.read_sql_query(
'''select *
from MODIFIED''', connection)
user = pd.DataFrame(SQL_Query1)

SQL_Query2 = pd.read_sql_query(
'''select *
from RESTAURANTS_1''', connection)
res = pd.DataFrame(SQL_Query2)

SQL_Query3 = pd.read_sql_query(
'''select *
from DS_D2''', connection)
data_p = pd.DataFrame(SQL_Query3)

In [234]:
if city != '-':
    l = list(d["Address"])
    city_list = []
    for i in l:
        j = []
        i = str(i)
        j = i.split(",")
        if len(j) > 1:
            city_list.append(j[-2].lstrip(" "))
        else:
            city_list.append(str(j).lstrip(" "))
    d["city"] = city_list
    fil = d[(d['city'] == city) |  (d.Name.isin(rest_name))]
else:
    def distance(lat,lon,destination1,destination2):
        lat1, lon1 = lat,lon
        lat2, lon2 = destination1,destination2
        radius = 6371 # km
        dlat = math.radians(lat2-lat1)
        dlon = math.radians(lon2-lon1)
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c
        return d

    d['Distance'] = d.apply(lambda x: distance(lat,lon , x.Latitude, x.Longitude), axis=1)
    fil = d[(d['Distance'] < 20) | (d.Name.isin(rest_name))]

In [181]:
fil_nam = list(fil['Name'])
user = pd.read_csv('/content/modified.csv')
res = pd.read_csv('/content/restaurants_1.csv')

boo = res.title.isin(fil_nam)
fil_res = res[boo]

boo1 = user.restaurant_id.isin(list(res.resId))
fil_user = user[boo1]

In [182]:
data_p = pd.read_csv("/content/Ds_d2.csv", usecols = ["Users", "Restaurant", "Ratings"])
boo2 = data_p.Restaurant.isin(fil_nam)
fil_data = data_p[boo2]

Cosine Similarity and weighted average

In [183]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [188]:
df=fil
df = df.reset_index()
df.Cuisine=df.Cuisine.fillna(" ")
tfv=TfidfVectorizer(min_df=3, max_features=None, 
                    strip_accents='unicode',analyzer='word',
                    token_pattern=r'\w{1,}', ngram_range=(1,3), stop_words='english')
tfv_matrix=tfv.fit_transform(df["Cuisine"])

In [189]:
cos=cosine_similarity(tfv_matrix, tfv_matrix)
ind=pd.Series(df.index, index=df["Name"]).drop_duplicates()
def give_rec_rest(rest, sig=cos):
    idx=ind[rest]
    sig_scores=list(enumerate(sig[idx])) #makes it a list keeping index in consideration
    print(sig_scores)
    sig_scores=sorted(sig_scores, key=lambda x:x[1], reverse=True)
    sig_scores=sig_scores[1:31]
    
    indi=[i[0] for i in sig_scores]
    return df.iloc[indi]

In [190]:
df["Tags"]=df["Tags"].replace("-"," ")
tfv_matrix1= tfv.fit_transform(df["Tags"])
cos1=cosine_similarity(tfv_matrix1,tfv_matrix1)
def give_rec_tag(resta, sig=cos1):
    idx = ind[resta]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    sig_scores = sig_scores[1:31]
    indi= [i[0] for i in sig_scores]
    return df.iloc[indi]

In [191]:
x = []
y = []
for i in range(len(rest_name)):
  df1 = give_rec_rest(rest_name[i],cos)
  x.extend(list(df1.Name))
  df2 = give_rec_tag(rest_name[i],cos1)
  y.extend(list(df2.Name))

[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 1.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.39811553761691837), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.0), (34, 0.39811553761691837), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.33995151866047474), (46, 0.39811553761691837), (47, 0.0), (48, 0.39811553761691837), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.0), (60, 0.0), (61, 0.0), (62, 0.0), (63, 0.0), (64, 0.0), (65, 0.0), (66, 0.0), (67, 0.0), (68, 0.0), (69, 0.0), (70, 0.0), (71, 0.0), (72, 0.39811553761691837), (73, 0.0), (74, 0.0), (75, 0.0), (76, 0.0), (77, 0.0), (78, 0.0), (79, 0.0), (80, 0.0), (81, 0.0), (82, 0.0), 

In [192]:
z = x+y
boo3 = d.Name.isin(z)
df3 =d[boo3]
df3 =df3.sort_values(by = 'Rating_weighted', ascending= False)
df3 = df3.reset_index()
df3.head()

,index,Unnamed: 0,Establishment_Type,Name,Rating,No_of_Reviews,Timings,Latitude,Longitude,Tags,Address,Contact_No,food,service,value,atmosphere,Rating_weighted,day_a,day_a1,time_a,time_a1,time_a2,time_a3,day_b,day_b1,time_b,time_b1,time_b2,time_b3,day_c,day_c1,time_c,time_c1,time_c2,time_c3,day_d,day_d1,time_d,time_d1,time_d2,time_d3,day_e,day_e1,time_e,time_e1,time_e2,time_e3,day_f,day_f1,time_f,time_f1,Cuisine,Price,city
0,101,101,Restaurant,Nikko Japanese Restaurant,5.0,73.0,"[{'days': 'Mon - Sat', 'times': ['11:00 AM - 9...",49.049930,-122.301200,-,"12-33324 South Fraser Way, Abbotsford, British...",+1 604-850-8968,4.5,4.5,4.5,4.0,4.486044,1.0,6.0,11.0,21.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"japanese, sushi, asian",highcost,Abbotsford
1,188,188,Restaurant,Brodeurs Bistro,4.5,816.0,"[{'days': 'Sun - Wed', 'times': ['11:00 AM - 1...",49.066475,-122.380360,"['All reviews ', 'smoked meat ', 'jambalaya ',...","3550 Mt Lehman Road, Abbotsford, British Colum...",+1 604-746-5900,4.5,4.5,4.5,4.0,4.455660,0.0,3.0,11.0,22.0,NaN,NaN,4.0,6.0,11.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"american, cajun creole, bar",highcost,Abbotsford
2,140,140,Restaurant,Greek Islands Restaurant,4.5,589.0,"[{'days': 'Tue - Sat', 'times': ['11:00 AM - 1...",49.051014,-122.330444,"['All reviews ', 'greek salad ', 'roast lamb '...","2686 Langdon St, Abbotsford, British Columbia ...",+1 604-850-1613,4.5,4.5,4.5,4.5,4.439759,2.0,6.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"mediterranean, greek, canadian",highcost,Abbotsford
3,103,103,Restaurant,Ann Maries Cafe,4.5,225.0,"[{'days': 'Mon - Fri', 'times': ['6:30 AM - 3:...",49.050340,-122.290000,"['All reviews ', 'poached eggs ', 'breakfast s...",33771 George Ferguson Way Parking behind off M...,+1 604-852-2610,4.5,4.5,4.5,4.5,4.358242,1.0,5.0,6.3,15.0,NaN,NaN,6.0,NaN,7.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"american, cafe, diner",highcost,Abbotsford
4,192,192,Restaurant,The Canadian Brewhouse & Grill,4.5,207.0,"[{'days': 'Mon - Thu', 'times': ['11:00 AM - 1...",49.058180,-122.380900,"['All reviews ', 'poutine ', 'chicken tenders ...","3122 Mt. Lehman Road E125, Abbotsford, British...",+1 604-625-5433,4.0,4.5,4.0,0.0,4.348076,1.0,4.0,11.0,0.0,NaN,NaN,5.0,NaN,11.0,1.0,NaN,NaN,6.0,0.0,10.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"bar, grill, pub",highcost,Abbotsford


In [193]:
result1 = list(df3.Name)

als and clustering

In [194]:
!pip install pyspark

In [195]:
import os
import time

# spark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction, explode, desc
from pyspark.sql.types import StringType, ArrayType
from pyspark.mllib.recommendation import ALS

# data science imports
import math
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans

%matplotlib inline

In [196]:
# spark config
spark = SparkSession \
    .builder \
    .appName("restaurant recommendation") \
    .config("spark.driver.maxResultSize", "96g") \
    .config("spark.driver.memory", "96g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.master", "local[12]") \
    .getOrCreate()
# get spark context
sc = spark.sparkContext

In [197]:
rest = spark.read.load(os.path.join('/content/restaurants_1.csv'), format='csv', header=True, inferSchema=True)
rest=rest.drop("_c0")

In [198]:
# load data
rest_rating = sc.textFile('/content/modified.csv')
header = rest_rating.take(1)[0]
rating_data = rest_rating \
    .filter(lambda line: line!=header) \
    .map(lambda line: line.split(",")) \
    .map(lambda tokens: (int(tokens[1]), int(tokens[2]), float(tokens[3]))) \
    .cache()
# check three rows

In [199]:
train, validation, test = rating_data.randomSplit([6, 2, 2], seed=99)

'''
def train_ALS(train_data, validation_data, num_iters, reg_param, ranks):
    """
    Grid Search Function to select the best model based on RMSE of hold-out data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in reg_param:
            # train ALS model
            model = ALS.train(
                ratings=train_data,    # (userID, productID, rating) tuple
                iterations=num_iters,
                rank=rank,
                lambda_=reg,           # regularization param
                seed=99)
            # make prediction
            valid_data = validation_data.map(lambda p: (p[0], p[1]))
            predictions = model.predictAll(valid_data).map(lambda r: ((r[0], r[1]), r[2]))
            # get the rating result
            ratesAndPreds = validation_data.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
            # get the RMSE
            MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
            error = math.sqrt(MSE)
            if error < min_error:
                min_error = error
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and regularization = {}'.format(best_rank, best_regularization))
    return best_model,best_rank,best_regularization

# hyper-param config
num_iterations = 10
ranks = [10, 14, 16, 18, 20]
reg_params = [0.005, 0.05, 0.1, 0.2]

# grid search and select best model
start_time = time.time()
final_model,best_rank,best_reg = train_ALS(train, validation, num_iterations, reg_params, ranks)
'''
best_rank,best_reg = 10,0.2

In [200]:
def get_resId(df_res, fav_res_list):
    resId_list = []
    for res in fav_res_list:
        resIds = df_res \
            .filter(rest.title.like('%{}%'.format(res))) \
            .select('resId') \
            .rdd \
            .map(lambda r: r[0]) \
            .collect()
        resId_list.extend(resIds)
    return list(set(resId_list))


def add_new_user_to_data(train_data, resId_list, spark_context):

    # get new user id
    new_id = train_data.map(lambda r: r[0]).max() + 1
    # get max rating
    max_rating = train_data.map(lambda r: r[2]).max()
    # create new user rdd
    user_rows = [(new_id, resId, max_rating) for resId in resId_list]
    new_rdd = spark_context.parallelize(user_rows)
    # return new train data
    return train_data.union(new_rdd)


def get_inference_data(train_data, df_res, resId_list):
    
    # get new user id
    new_id = train_data.map(lambda r: r[0]).max() + 1
    # return inference rdd
    return df_res.rdd \
        .map(lambda r: r[0]) \
        .distinct() \
        .filter(lambda x: x not in resId_list) \
        .map(lambda x: (new_id, x))


def make_recommendation(best_model_params, ratings_data, df_res, 
                        fav_movie_list, n_recommendations, spark_context):
    
    # modify train data by adding new user's rows
    resId_list = get_resId(df_res, fav_movie_list)
    train_data = add_new_user_to_data(ratings_data, resId_list, spark_context)
    
    # train best ALS
    model = ALS.train(
        ratings=train_data,
        iterations=best_model_params.get('iterations', None),
        rank=best_model_params.get('rank', None),
        lambda_=best_model_params.get('lambda_', None),
        seed=99)
    
    # get inference rdd
    inference_rdd = get_inference_data(ratings_data, df_res, resId_list)
    
    # inference
    predictions = model.predictAll(inference_rdd).map(lambda r: (r[1], r[2]))
    
    # get top n movieId
    topn_rows = predictions.sortBy(lambda r: r[1], ascending=False).take(n_recommendations)
    topn_ids = [r[0] for r in topn_rows]
    
    # return movie titles
    return df_res.filter(rest.resId.isin(topn_ids)) \
                    .select('title') \
                    .rdd \
                    .map(lambda r: r[0]) \


In [201]:
data = fil
data['Cuisine'] = data['Cuisine'].fillna(" ")
import re
def remove_punc(text):
    text = text.replace(',', ' ')
    return text

data['Cuisine'] = data['Cuisine'].apply(lambda x : remove_punc(x))
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

tfv_matrix = tfv.fit_transform(data['Cuisine'])  
df = pd.DataFrame(tfv_matrix[0].T.todense(), index=tfv.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [202]:
kmeans = KMeans(n_clusters=50,init='k-means++',random_state=0)
kmeans.fit(tfv_matrix)
data['clusters'] = kmeans.labels_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [203]:
def cluster_predict(str_input):
    Y = tfv.transform(str_input)
    prediction = kmeans.predict(Y)
    return prediction

In [204]:
user = fil_user
res = fil_res
users_id = list(user['user_id'])

In [205]:
new_user = usid
fav_cus = fav_cuisine

if new_user not in users_id:
  print('new user')
  f = fav_cus
  prediction = cluster_predict(f)
  data1 =  data[data['clusters'] == prediction[0]]
  list2 = list(data1['Name'])

  my_favorite_restaurants = list2[:50]
  # get recommends
  r2 = recommends = make_recommendation(
      best_model_params={'iterations': 10, 'rank': best_rank, 'lambda_': best_reg}, 
      ratings_data=rating_data, 
      df_res=rest, 
      fav_movie_list=my_favorite_restaurants, 
      n_recommendations=50, 
      spark_context=sc)
  
  result2 = list2[:7]
  for i in r2.collect():
    result2.append(i)

else:
  print('existing user')
  x = list(user[(user['user_id'] == new_user) & (user['Rating'] > 3.0)]['restaurant_id'])
  my_favorite_restaurants = [(res[res['resId'] == i]['title'].values)[0] for i in x]

  r2 = recommends = make_recommendation(
      best_model_params={'iterations': 10, 'rank': best_rank, 'lambda_': best_reg}, 
      ratings_data=rating_data, 
      df_res=rest, 
      fav_movie_list=my_favorite_restaurants, 
      n_recommendations=10, 
      spark_context=sc)
  
  result2 = []
  for i in r2.collect():
    result2.append(i)

  cus = [(data[data['Name'] == i]['Cuisine'].values)[0] for i in result2[:3]]
  prediction = cluster_predict(cus)
  data1 =  data[data['clusters'] == prediction[0]]
  list2 = list(data['Name'])
  result2 += list2

result2

new user


["Paliotti's Italian Restaurants",
 "Antonio's Restaurant",
 'Pizza Garden',
 'Subway',
 "McDonald's",
 'Tim Hortons',
 "Denny's",
 'Ravens Bar & Grill',
 'Mandarin Garden',
 "Carl's Jr.",
 'Wings',
 'Curry Time',
 'Hot Chili House',
 'KFC',
 'Zaika',
 'A&W Restaurant',
 'Wok Box',
 "O'Neill's Home Cooking",
 "Bob's Bar and Grill",
 "Suzy's Cafe",
 'Islands Bar',
 'Tandoori Bites',
 'The Great Wok',
 'Dusty Rose Pub',
 'Mirchi Restaurant',
 'Nirvana',
 'IHOP',
 'Sidharth Grill',
 'Cafe Mai-Mai & Pho',
 'Desi Junction',
 'Bombay Sweets',
 'Mr. Red Cafe',
 'Joyeaux Cafe & Restaurant',
 'Canoe Beach Cafe',
 'Cafe Tasse',
 'UniTea Cafe & Lounge',
 'Bon Cafe',
 "Margo's Cafe"]

Surprise

In [206]:
!pip install surprise

In [207]:
import surprise
from surprise import Dataset, Reader
from surprise.similarities import pearson, pearson_baseline
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate
from surprise import accuracy

In [208]:
import pandas as pd
ids = fil_user

res_id = []
for i in ids["restaurant_id"]:
    res_id.append(i)

users = []
for i in ids["user_id"]:
    users.append(i)

data = pd.read_csv("Ds_d2.csv", usecols = ["Users", "Restaurant", "Ratings"])
data.head()
data["restaurant_id"] = res_id
data["user_id"] = users

reader = Reader(line_format='user item rating', sep=',', rating_scale = (1,10))
tot_data = Dataset.load_from_df(data[["user_id", "restaurant_id", "Ratings"]], reader=reader)

trainset, testset = train_test_split(tot_data, test_size=0.2)

trainset_iids = list(trainset.all_items())
iid_converter = lambda x: trainset.to_raw_iid(x)
trainset_raw_iids = list(map(iid_converter, trainset_iids))

trainsetfull = tot_data.build_full_trainset()

trainsetfull_iids = list(trainsetfull.all_items())
iid_converter = lambda x: trainsetfull.to_raw_iid(x)
trainsetfull_raw_iids = list(map(iid_converter, trainsetfull_iids))

sim_options = {"name":"pearson", "user_based":False}

In [209]:
def return_top_similar_dataframe(similarity_matrix, raw_ids, top_x):
    length = similarity_matrix.shape[0]
    closest_ids = np.zeros((top_x,length))
    for item in range(0,length):    
        similarity_metrics = similarity_matrix[item]        
        sorted_metrics, sorted_raw_ids = zip(*sorted(zip(similarity_metrics, raw_ids)))        
        for index in range(0, top_x):
            closest_ids[index][item] = sorted_raw_ids[-2-index]
        similarity_df = pd.DataFrame()
        similarity_df['restaurant_id'] = raw_ids
        for index in range(0, top_x): 
            similarity_df['similar_res_' + str(index + 1)] = closest_ids[index].astype(int)
    return similarity_df

id_to_name_dict = {}
res_names = data["Restaurant"]
res_ids = data["restaurant_id"]
for index in range(0, len(res_ids)):
    id_to_name_dict[str(res_ids[index])] = res_names[index]

def get_res_name_from_id(res_id):
    return id_to_name_dict[str(res_id)]

def save_similar_res(similarity_matrix, raw_iids, top_x):
    sim_df = return_top_similar_dataframe(similarity_matrix, raw_iids, top_x)
    for column in sim_df.columns:
        for i in range(len(sim_df[column])):
            sim_df[column][i] = get_res_name_from_id(sim_df[column][i])
    return sim_df

from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans, KNNBaseline
model1 = KNNBasic(sim_options = sim_options, verbose = False)
model1.fit(trainsetfull)
model2 = KNNWithMeans(sim_options = sim_options, verbose = False)
model2.fit(trainsetfull)
model3 = KNNBaseline(k = 30, sim_options = sim_options, bsl_options = {"method":"sgd","learning_rate":0.00006})
model3.fit(trainsetfull)
sim_basic_df = save_similar_res(model1.sim, trainsetfull_raw_iids, 30)
sim_means_df = save_similar_res(model2.sim, trainsetfull_raw_iids, 30)
sim_base_df = save_similar_res(model3.sim, trainsetfull_raw_iids, 30)

results_basic = cross_validate(model1, tot_data, measures=['RMSE'], cv=3, return_train_measures=True);
results_means = cross_validate(model2, tot_data, measures=['RMSE'], cv=3, return_train_measures=True);
results_base = cross_validate(model3, tot_data, measures=['RMSE'], cv = 3, return_train_measures=True)

Estimating biases using sgd...
Computing the pearson similarity matrix...
Done computing similarity matrix.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Estimating biases using sgd...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [210]:
result3 = []
for i in range(len(rest_name)):
  if rest_name in list(res_names):
    rec3 = sim_basic_df[sim_basic_df["restaurant_id"] == res_name]
    result3.extend(list(rec3.iloc[0, 1:]))

In [219]:
result = {}
if len(result3) == 0:
  p = list(set(result1+result2))
  for i in p:
    if i in result1 and i in result2:
      x = result1.index(i)
      y = result2.index(i)
      t = (0.6 * x) + (0.4 * y)
    elif i in result1:
      x = result1.index(i)
      t = 0.6 * x
    elif i in result2:
      y = result2.index(i)
      t = 0.4 * y

    result[t] = i
else:
  p = list(set(result1+result2))
  for i in p:
    x = 0
    y = 0
    z = 0
    if i in result1:
      x = result1.index(i)
    if i in result2:
      y = result1.index(i)
    if i in result3:
      z = result1.index(i)
    
    t = (0.3 * x) + (0.4 * y) + (0.3 * z)
    result[t] = i

In [220]:
x = sorted(list(result.keys()))
results = [result[x[i]] for i in range(len(x))]
results[:10]

['Nikko Japanese Restaurant',
 "Antonio's Restaurant",
 'Brodeurs Bistro',
 'Pizza Garden',
 'Greek Islands Restaurant',
 'Ann Maries Cafe',
 'Tim Hortons',
 'The Canadian Brewhouse & Grill',
 "Denny's",
 'Ravens Bar & Grill']

In [222]:
idx = pd.Index(d['Name']).get_indexer(results)
Result = d.iloc[idx]
if city != '-':
  Result = Result[Result['city'] == city]
else:
  Result = Result[Result['Distance'] < 20]
Result = Result.sort_values(by = order_pref[0], ascending = False)
Result.head()

,Unnamed: 0,Establishment_Type,Name,Rating,No_of_Reviews,Timings,Latitude,Longitude,Tags,Address,Contact_No,food,service,value,atmosphere,Rating_weighted,day_a,day_a1,time_a,time_a1,time_a2,time_a3,day_b,day_b1,time_b,time_b1,time_b2,time_b3,day_c,day_c1,time_c,time_c1,time_c2,time_c3,day_d,day_d1,time_d,time_d1,time_d2,time_d3,day_e,day_e1,time_e,time_e1,time_e2,time_e3,day_f,day_f1,time_f,time_f1,Cuisine,Price,city
46,46,Restaurant,Maru Sushi,4.5,22.0,"[{'days': 'Sun', 'times': ['11:30 AM - 9:00 PM...",49.036210,-122.230410,-,"124-1900 North Parallel Rd, Abbotsford, Britis...",+1 604-744-1105,5.0,5.0,4.5,0.0,3.922230,0.0,NaN,11.3,21.0,NaN,NaN,1.0,6.0,11.3,21.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"japanese, sushi, asian",highcost,Abbotsford
65,65,Restaurant,Meat & Bread,5.0,11.0,"[{'days': 'Mon - Fri', 'times': ['11:00 AM - 3...",49.046375,-122.287224,"['All reviews ', 'lunch ', 'porchetta ', 'sand...","2455 West Railway St, Abbotsford, British Colu...",+1 604-566-9003,5.0,4.5,5.0,0.0,3.906676,1.0,5.0,11.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"fast food, british, canadian",highcost,Abbotsford
213,213,Restaurant,U and I Thai Fine Cuisine,4.5,20.0,"[{'days': 'Mon - Sat', 'times': ['11:30 AM - 9...",49.047590,-122.289840,-,"33757 South Fraser Way, Abbotsford, British Co...",+1 604-744-0488,5.0,4.5,4.5,0.0,3.904175,1.0,6.0,11.3,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"asian, taiwanese, vegetarian friendly",highcost,Abbotsford
87,87,Restaurant,Sasha Sushi,4.5,16.0,NaN,49.039590,-122.270360,"['All reviews ', 'bento box ', 'sushi ', 'grea...","2070 Sumas Way #108, Abbotsford, British Colum...",+1 604-852-7874,5.0,5.0,4.5,0.0,3.864453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"japanese, sushi, asian",highcost,Abbotsford
218,218,Restaurant,Bhav's Indian Cuisine,5.0,10.0,NaN,49.049538,-122.335700,-,"2591 Cedar Park Pl, Abbotsford, British Columb...",+1 604-557-9492,5.0,4.5,4.5,0.0,3.886429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vegetarian friendly,highcost,Abbotsford


In [217]:
rest_name = ["Koya Japan", "An Indian Affair"]
o  = d[d['Name'] == "Koya Japan"]
o.head()

,Unnamed: 0,Establishment_Type,Name,Rating,No_of_Reviews,Timings,Latitude,Longitude,Tags,Address,Contact_No,food,service,value,atmosphere,Rating_weighted,day_a,day_a1,time_a,time_a1,time_a2,time_a3,day_b,day_b1,time_b,time_b1,time_b2,time_b3,day_c,day_c1,time_c,time_c1,time_c2,time_c3,day_d,day_d1,time_d,time_d1,time_d2,time_d3,day_e,day_e1,time_e,time_e1,time_e2,time_e3,day_f,day_f1,time_f,time_f1,Cuisine,Price,city
9,9,Restaurant,Koya Japan,0.0,0.0,NaN,49.04936,-122.31277,-,"32900 South Fraser Way, Abbotsford, British Co...",+1 604-855-4177,0.0,0,0.0,0.0,3.633345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'japanese',not_available,Abbotsford


In [218]:
o  = d[d['Name'] == "An Indian Affair"]
o.head()

,Unnamed: 0,Establishment_Type,Name,Rating,No_of_Reviews,Timings,Latitude,Longitude,Tags,Address,Contact_No,food,service,value,atmosphere,Rating_weighted,day_a,day_a1,time_a,time_a1,time_a2,time_a3,day_b,day_b1,time_b,time_b1,time_b2,time_b3,day_c,day_c1,time_c,time_c1,time_c2,time_c3,day_d,day_d1,time_d,time_d1,time_d2,time_d3,day_e,day_e1,time_e,time_e1,time_e2,time_e3,day_f,day_f1,time_f,time_f1,Cuisine,Price,city
252,252,Restaurant,An Indian Affair,4.5,390.0,"[{'days': 'Sun - Thu', 'times': ['11:00 AM - 9...",49.11562,-122.67953,"['All reviews ', 'naan ', 'butter chicken ', '...","146-19653 Willowbrook Dr, British Columbia V2Y...",+1 604-539-8114,4.5,4.5,4.5,4.5,4.412136,0.0,4.0,11.0,21.3,NaN,NaN,5.0,6.0,11.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"indian, international, vegetarian friendly",highcost,146-19653 Willowbrook Dr
